In [ ]:
from collections.abc import Sequence
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

text = "this is a sample text to demonstrate next word prediction "

# Call the split method to get the tokens
tokens = text.lower().split()
vocab = sorted(list(set(tokens)))
word_to_index = {word: index for index, word in enumerate(vocab)}
index_to_word = {index: word for index, word in enumerate(vocab)}
vocab_size = len(vocab)

# create input sequences and labels
Sequence_length = 3
sequences = []
labels = []
for i in range(len(tokens) - Sequence_length):
    sequence = tokens[i:i + Sequence_length]
    label = tokens[i + Sequence_length]
    sequences.append([word_to_index[word] for word in sequence])
    labels.append(word_to_index[label])

# convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

# define the model
model = Sequential()
# Remove input_length argument from Embedding layer
model.add(Embedding(vocab_size, 10))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))

# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(sequences, labels, epochs=50, verbose=0)

# function to predict the next word
def predict_next_word(input_text):
    input_seq = [word_to_index[word] for word in input_text.lower().split()[-Sequence_length:]]
    input_seq = input_seq + [0] * (Sequence_length - len(input_seq))
    input_seq = np.array([input_seq])
    predicted_index = np.argmax(model.predict(input_seq, verbose=0))
    return index_to_word[predicted_index]

# example usage
input_text = "this is a "
predicted_word = predict_next_word(input_text)
print(f"Input: {input_text}, Predicted: {predicted_word}")

Input: this is a , Predicted: sample


In [ ]:
!pip install regex

In [ ]:
import re
from collections import defaultdict
class SimpleTokenizer:
  def _init_(self):
    self.vocab={}
    self.inv_vocab={}
    self.index=0

  def fit(self, text):
    tokens=set(re.findall(r'\w+', text))
    self.vocab={token:idx for idx, token in enumerate(tokens)}
    self.inv_vocab={idx:token for token, idx in self.vocab.items()}
  def encode(self, text):
    return [self.vocab[token] for token in re.findall(r'\w+', text)]
  def decode(self,indices):
    return ' '.join([self.inv_vocab[idx] for idx in indices])
#example usage
text="hello world! This is ssimple tokenizer"
tokenizer=SimpleTokenizer()
tokenizer.fit(text)
tokens=tokenizer.encode(text)
print("Tokens:",tokens)




Tokens: [0, 4, 3, 2, 1, 5]


In [ ]:
import numpy as np
class SimpleEmbedding:
  def __init__(self, vocab_size, embed_size): # Changed _init_ to __init__
    self.embeddings = np.random.randn(vocab_size, embed_size)
  def get_embedding(self, token_id):
    return self.embeddings[token_id]
#example usage
vocab_size = len(tokenizer.vocab)
embed_size = 10
embedding=SimpleEmbedding(vocab_size, embed_size)
embedded_tokens=np.array([embedding.get_embedding(token) for token in tokens])
print("Embedded Tokens Shape:",embedded_tokens.shape)

Embedded Tokens Shape: (6, 10)


In [ ]:
def self_attention(q,k,v):
  attention_scores=np.dot(q,k.T)/np.sqrt(q.shape[-1])
  output=np.dot(attention_scores,v)
  return output,attention_scores
q=k=v=embedded_tokens
output,attention_probs=self_attention(q,k,v)
print(output.shape)

(6, 10)


In [ ]:
class TransformerDecoderLayer:
  def __init__(self, embed_size):
    self.embed_size=embed_size
    #initialize weights for linear transformations
    self.w1=np.random.randn(embed_size, embed_size)
    self.w2=np.random.randn(embed_size, embed_size)
  def forward(self,x):
    #apply self-attention mechanism
    attn_output,_=self_attention(x,x,x)
    #apply linear transformation
    attn_output =np.dot(attn_output, self.w1)
    output=np.dot(attn_output,self.w2)
    return output
#example usage
decoder_layer=TransformerDecoderLayer(embed_size)
decoder_output=decoder_layer.forward(embedded_tokens)
print(decoder_output.shape)

(6, 10)


In [ ]:
def softmax( x):
  exp_x=np.exp(x-np.max(x,axis=-1,keepdims=True))
  return exp_x/np.sum(exp_x,axis=-1,keepdims=True)
class SimpleLanguageModel:
  def __init__(self,vocab_size,embed_size): # Changed init to __init__
    self.embedding=SimpleEmbedding(vocab_size,embed_size)
    self.decoder_layer=TransformerDecoderLayer(embed_size)
    self.output_layer=np.random.randn(embed_size,vocab_size)
  def predict(self,input_sequence):
    embedded_tokens=np.array([self.embedding.get_embedding(token) for token in input_sequence])
    decoder_output=self.decoder_layer.forward(embedded_tokens)
    logits=np.dot(decoder_output,self.output_layer)
    probs=softmax(logits)
    return probs
#example usage
model=SimpleLanguageModel(vocab_size,embed_size)
probs=model.predict(tokens)
predicted_token=np.argmax(probs,axis=1)
print(predicted_token)
print(tokenizer.decode(predicted_token))

[2 4 2 2 2 4]
is world is is is world


In [ ]:
!pip install openai


In [ ]:
!pip install pandas